In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# 필요한 라이브러리 임포트
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# ---------- Dice Loss 함수 정의 ----------
# 예측과 실제 정답 간의 겹치는 정도를 측정하는 Dice Loss 함수
def dice_loss(pred, target, smooth=1e-6):
    pred = torch.softmax(pred, dim=1)[:, 1, :, :]  # 클래스 1에 대한 softmax 확률
    target = (target == 1).float()                 # target도 1인 픽셀만 선택
    intersection = (pred * target).sum(dim=(1, 2)) # 교집합
    union = pred.sum(dim=(1, 2)) + target.sum(dim=(1, 2)) # 합집합
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice.mean()

# ---------- 데이터셋 클래스 ----------
class LaneDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir      # 이미지 디렉토리
        self.mask_dir = mask_dir    # 마스크 디렉토리
        self.images = sorted(os.listdir(img_dir))  # 이미지 파일 정렬
        self.transform = transform

    def __len__(self): 
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        pil_img = Image.open(img_path).convert('RGB')
        original_size = pil_img.size  # (너비, 높이)
        image = self.transform(pil_img)

        if self.mask_dir:
            # 학습/검증 시 마스크도 함께 불러옴
            mask_path = os.path.join(self.mask_dir, img_name)
            mask = Image.open(mask_path).convert('L').resize((1024, 512))
            mask = (np.array(mask) > 127).astype(np.uint8)  # 이진화
            mask = torch.tensor(mask, dtype=torch.long)
            return image, mask
        else:
            # 테스트 시 마스크 없음 → 파일명과 원본 크기 반환
            return image, img_name, original_size

# ---------- 간단한 CNN 모델 정의 ----------
class SimpleCNNModel(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        # 인코더 (특징 추출)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )
        # 디코더 (업샘플링)
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 2, 2), nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 2, 2), nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1), nn.ReLU()
        )
        # 클래스 분류기
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return self.classifier(x)

# ---------- 마스크를 RLE로 인코딩하는 함수 ----------
def mask_to_rle(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# ---------- 메인 함수 ----------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 이미지 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])

    # --- 학습 시작 ---
    train_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/train/train/frames",
        "/kaggle/input/lanesegmentationchallenge/train/train/lane-masks",
        transform
    )
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

    model = SimpleCNNModel(n_class=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    ce_loss_fn = nn.CrossEntropyLoss()

    print("▶️ 모델 학습 시작...")
    for epoch in range(1):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for imgs, masks in pbar:
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            ce = ce_loss_fn(outputs, masks)
            d  = dice_loss(outputs, masks)
            loss = 0.5 * ce + 0.5 * d  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=f"{loss.item():.4f}")
        print(f"[Epoch {epoch+1}] 평균 손실: {total_loss / len(train_loader):.4f}")

    torch.save(model.state_dict(), "simple_cnn_model.pth")
    print("💾 모델이 simple_cnn_model.pth로 저장되었습니다.")

    # --- 테스트 이미지에 대한 추론 및 제출 파일 생성 ---
    print("🧪 submission.csv 생성 시작...")
    test_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/test/test/frames",
        None, transform
    )
    test_loader = DataLoader(test_dataset, batch_size=1)
    model.eval()

    results = []
    with torch.no_grad():
        for imgs, fnames, orig_size in tqdm(test_loader, desc="추론 중"):
            imgs = imgs.to(device)
            output = model(imgs)
            pred_mask = output.argmax(1).squeeze(0).cpu().numpy().astype(np.uint8)
            pred_pil = Image.fromarray(pred_mask)
            W, H = orig_size
            resized_mask = pred_pil.resize((W, H), resample=Image.NEAREST)
            binary = np.array(resized_mask) > 0
            rle = mask_to_rle(binary)
            results.append({'filename': fnames[0], 'rle': rle})

    pd.DataFrame(results).to_csv("submission.csv", index=False)
    print("✅ submission.csv 파일이 저장되었습니다!")

# 프로그램 실행 진입점
if __name__ == "__main__":
    main()


▶️ 모델 학습 시작...


Epoch 1: 100%|██████████| 454/454 [04:44<00:00,  1.60it/s, loss=0.4136]


[Epoch 1] 평균 손실: 0.5443
💾 모델이 simple_cnn_model.pth로 저장되었습니다.
🧪 submission.csv 생성 시작...


추론 중: 100%|██████████| 2782/2782 [02:24<00:00, 19.25it/s]


✅ submission.csv 파일이 저장되었습니다!


In [16]:
torch.cuda.empty_cache()

In [5]:
# 필요한 라이브러리 임포트
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# ---------- Dice Loss 함수 정의 ----------
# 예측과 실제 정답 간의 겹치는 정도를 측정하는 Dice Loss 함수
def dice_loss(pred, target, smooth=1e-6):
    pred = torch.softmax(pred, dim=1)[:, 1, :, :]  # 클래스 1에 대한 softmax 확률
    target = (target == 1).float()                 # target도 1인 픽셀만 선택
    intersection = (pred * target).sum(dim=(1, 2)) # 교집합
    union = pred.sum(dim=(1, 2)) + target.sum(dim=(1, 2)) # 합집합
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice.mean()

# ---------- 데이터셋 클래스 ----------
class LaneDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir      # 이미지 디렉토리
        self.mask_dir = mask_dir    # 마스크 디렉토리
        self.images = sorted(os.listdir(img_dir))  # 이미지 파일 정렬
        self.transform = transform

    def __len__(self): 
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        pil_img = Image.open(img_path).convert('RGB')
        original_size = pil_img.size  # (너비, 높이)
        image = self.transform(pil_img)

        if self.mask_dir:
            # 학습/검증 시 마스크도 함께 불러옴
            mask_path = os.path.join(self.mask_dir, img_name)
            mask = Image.open(mask_path).convert('L').resize((1024, 512))
            mask = (np.array(mask) > 127).astype(np.uint8)  # 이진화
            mask = torch.tensor(mask, dtype=torch.long)
            return image, mask
        else:
            # 테스트 시 마스크 없음 → 파일명과 원본 크기 반환
            return image, img_name, original_size

# ---------- 간단한 CNN 모델 정의 ----------
class SimpleCNNModel(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        # 인코더 (특징 추출)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )
        # 디코더 (업샘플링)
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 2, 2), nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 2, 2), nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1), nn.ReLU()
        )
        # 클래스 분류기
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return self.classifier(x)
# ---
class VGG16Seg(nn.Module):
    def __init__(self, n_class=2, pretrained=True):
        super().__init__()
        vgg = models.vgg16(pretrained=pretrained)
        self.encoder = vgg.features
        self.classifier = nn.Conv2d(512, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)  # [B, n_class, H/32, W/32]
        # Upsample back to input size
        return nn.functional.interpolate(x, scale_factor=32, mode='bilinear', align_corners=False)
# ---------- 마스크를 RLE로 인코딩하는 함수 ----------
def mask_to_rle(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# ---------- 메인 함수 ----------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 이미지 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])

    # --- 학습 시작 ---
    train_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/train/train/frames",
        "/kaggle/input/lanesegmentationchallenge/train/train/lane-masks",
        transform
    )
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

    model = VGG16Seg(n_class=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    ce_loss_fn = nn.CrossEntropyLoss()

    print("▶️ 모델 학습 시작...")
    for epoch in range(1):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for imgs, masks in pbar:
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            ce = ce_loss_fn(outputs, masks)
            d  = dice_loss(outputs, masks)
            loss = 0.5 * ce + 0.5 * d  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=f"{loss.item():.4f}")
        print(f"[Epoch {epoch+1}] 평균 손실: {total_loss / len(train_loader):.4f}")

    torch.save(model.state_dict(), "simple_cnn_model.pth")
    print("💾 모델이 simple_cnn_model.pth로 저장되었습니다.")

    # --- 테스트 이미지에 대한 추론 및 제출 파일 생성 ---
    print("🧪 submission.csv 생성 시작...")
    test_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/test/test/frames",
        None, transform
    )
    test_loader = DataLoader(test_dataset, batch_size=1)
    model.eval()

    results = []
    with torch.no_grad():
        for imgs, fnames, orig_size in tqdm(test_loader, desc="추론 중"):
            imgs = imgs.to(device)
            output = model(imgs)
            pred_mask = output.argmax(1).squeeze(0).cpu().numpy().astype(np.uint8)
            pred_pil = Image.fromarray(pred_mask)
            W, H = orig_size
            resized_mask = pred_pil.resize((W, H), resample=Image.NEAREST)
            binary = np.array(resized_mask) > 0
            rle = mask_to_rle(binary)
            results.append({'filename': fnames[0], 'rle': rle})

    pd.DataFrame(results).to_csv("submission.csv", index=False)
    print("✅ submission.csv 파일이 저장되었습니다!")

# 프로그램 실행 진입점
if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 222MB/s] 


▶️ 모델 학습 시작...


Epoch 1: 100%|██████████| 454/454 [05:15<00:00,  1.44it/s, loss=0.2645]


[Epoch 1] 평균 손실: 0.3075
💾 모델이 simple_cnn_model.pth로 저장되었습니다.
🧪 submission.csv 생성 시작...


추론 중: 100%|██████████| 2782/2782 [02:30<00:00, 18.48it/s]


✅ submission.csv 파일이 저장되었습니다!


In [7]:
# 필요한 라이브러리 임포트
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# ---------- Dice Loss 함수 정의 ----------
# 예측과 실제 정답 간의 겹치는 정도를 측정하는 Dice Loss 함수
def dice_loss(pred, target, smooth=1e-6):
    pred = torch.softmax(pred, dim=1)[:, 1, :, :]  # 클래스 1에 대한 softmax 확률
    target = (target == 1).float()                 # target도 1인 픽셀만 선택
    intersection = (pred * target).sum(dim=(1, 2)) # 교집합
    union = pred.sum(dim=(1, 2)) + target.sum(dim=(1, 2)) # 합집합
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice.mean()

# ---------- 데이터셋 클래스 ----------
class LaneDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir      # 이미지 디렉토리
        self.mask_dir = mask_dir    # 마스크 디렉토리
        self.images = sorted(os.listdir(img_dir))  # 이미지 파일 정렬
        self.transform = transform

    def __len__(self): 
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        pil_img = Image.open(img_path).convert('RGB')
        original_size = pil_img.size  # (너비, 높이)
        image = self.transform(pil_img)

        if self.mask_dir:
            # 학습/검증 시 마스크도 함께 불러옴
            mask_path = os.path.join(self.mask_dir, img_name)
            mask = Image.open(mask_path).convert('L').resize((1024, 512))
            mask = (np.array(mask) > 127).astype(np.uint8)  # 이진화
            mask = torch.tensor(mask, dtype=torch.long)
            return image, mask
        else:
            # 테스트 시 마스크 없음 → 파일명과 원본 크기 반환
            return image, img_name, original_size

# ---------- 간단한 CNN 모델 정의 ----------
class SimpleCNNModel(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        # 인코더 (특징 추출)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )
        # 디코더 (업샘플링)
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 2, 2), nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 2, 2), nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1), nn.ReLU()
        )
        # 클래스 분류기
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return self.classifier(x)
# ---
class VGG16Seg(nn.Module):
    def __init__(self, n_class=2, pretrained=True):
        super().__init__()
        vgg = models.vgg16(pretrained=pretrained)
        self.encoder = vgg.features
        self.classifier = nn.Conv2d(512, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)  # [B, n_class, H/32, W/32]
        # Upsample back to input size
        return nn.functional.interpolate(x, scale_factor=32, mode='bilinear', align_corners=False)
# ---------- 마스크를 RLE로 인코딩하는 함수 ----------
class RESNET16Seg(nn.Module):
    def __init__(self, n_class=2, pretrained=True):
        super().__init__()
        resnet = models.resnet18(pretrained=pretrained)
        self.encoder = nn.Sequential(*list(resnet.children())[:-2])
        self.classifier = nn.Conv2d(512, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)  # [B, n_class, H/32, W/32]
        # Upsample back to input size
        return nn.functional.interpolate(x, scale_factor=32, mode='bilinear', align_corners=False)
#--
def mask_to_rle(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# ---------- 메인 함수 ----------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 이미지 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])

    # --- 학습 시작 ---
    train_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/train/train/frames",
        "/kaggle/input/lanesegmentationchallenge/train/train/lane-masks",
        transform
    )
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

    model = RESNET16Seg(n_class=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    ce_loss_fn = nn.CrossEntropyLoss()

    print("▶️ 모델 학습 시작...")
    for epoch in range(1):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for imgs, masks in pbar:
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            ce = ce_loss_fn(outputs, masks)
            d  = dice_loss(outputs, masks)
            loss = 0.5 * ce + 0.5 * d  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=f"{loss.item():.4f}")
        print(f"[Epoch {epoch+1}] 평균 손실: {total_loss / len(train_loader):.4f}")

    torch.save(model.state_dict(), "simple_cnn_model.pth")
    print("💾 모델이 simple_cnn_model.pth로 저장되었습니다.")

    # --- 테스트 이미지에 대한 추론 및 제출 파일 생성 ---
    print("🧪 submission.csv 생성 시작...")
    test_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/test/test/frames",
        None, transform
    )
    test_loader = DataLoader(test_dataset, batch_size=1)
    model.eval()

    results = []
    with torch.no_grad():
        for imgs, fnames, orig_size in tqdm(test_loader, desc="추론 중"):
            imgs = imgs.to(device)
            output = model(imgs)
            pred_mask = output.argmax(1).squeeze(0).cpu().numpy().astype(np.uint8)
            pred_pil = Image.fromarray(pred_mask)
            W, H = orig_size
            resized_mask = pred_pil.resize((W, H), resample=Image.NEAREST)
            binary = np.array(resized_mask) > 0
            rle = mask_to_rle(binary)
            results.append({'filename': fnames[0], 'rle': rle})

    pd.DataFrame(results).to_csv("submission.csv", index=False)
    print("✅ submission.csv 파일이 저장되었습니다!")

# 프로그램 실행 진입점
if __name__ == "__main__":
    main()


▶️ 모델 학습 시작...


Epoch 1: 100%|██████████| 454/454 [01:33<00:00,  4.87it/s, loss=0.2442]


[Epoch 1] 평균 손실: 0.3259
💾 모델이 simple_cnn_model.pth로 저장되었습니다.
🧪 submission.csv 생성 시작...


추론 중: 100%|██████████| 2782/2782 [01:35<00:00, 29.21it/s]


✅ submission.csv 파일이 저장되었습니다!


In [19]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# ---------- Dice Loss 함수 정의 ----------
def dice_loss(pred, target, smooth=1e-6):
    pred_prob = torch.sigmoid(pred)
    target = target.float()
    intersection = (pred_prob * target).sum(dim=(1, 2))
    union = pred_prob.sum(dim=(1, 2)) + target.sum(dim=(1, 2))
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice.mean()

# ---------- 데이터셋 클래스 ----------
class LaneDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = sorted(os.listdir(img_dir))
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        pil_img = Image.open(img_path).convert('RGB')
        original_size = pil_img.size
        image = self.transform(pil_img)

        if self.mask_dir:
            mask_path = os.path.join(self.mask_dir, img_name)
            mask = Image.open(mask_path).convert('L').resize((1024, 512))
            mask = (np.array(mask) > 127).astype(np.uint8)
            mask = torch.tensor(mask, dtype=torch.float32)
            return image, mask
        else:
            return image, img_name, original_size

# ---------- 간단한 CNN 모델 정의 ----------
class SimpleCNNModel(nn.Module):
    def __init__(self, n_class=1):
        super().__init__()
        # 인코더
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 128, 3, padding=1), nn.ReLU()
        )
        # 디코더
        self.decoder = nn.Sequential(
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 2, 2), nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 2, 2), nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1), nn.ReLU()
        )
        # 분류기
        self.classifier = nn.Conv2d(32, 1, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return self.classifier(x).squeeze(1)

# ---------- Semi-Supervised 학습 함수 ----------
def train_semi(model, labeled_loader, unlabeled_loader, optimizer, device, lambda_u=0.5):
    model.train()
    total_loss = 0.0
    unl_iter = iter(unlabeled_loader)
    bce = nn.BCEWithLogitsLoss()
    for imgs_l, masks in tqdm(labeled_loader, desc='Semi-Train'):
        imgs_l, masks = imgs_l.to(device), masks.to(device)
        # Supervised loss
        out_l = model(imgs_l)
        loss_sup = bce(out_l, masks) + dice_loss(out_l, masks)
        # Unsupervised pseudo-labeling
        try:
            imgs_u, _, _ = next(unl_iter)
        except StopIteration:
            unl_iter = iter(unlabeled_loader)
            imgs_u, _, _ = next(unl_iter)
        imgs_u = imgs_u.to(device)
        with torch.no_grad():
            pseudo = torch.sigmoid(model(imgs_u))
        mask_p = (pseudo > 0.9).float()
        loss_unsup = bce(model(imgs_u), mask_p)
        # Total loss
        loss = loss_sup + lambda_u * loss_unsup
        model.zero_grad()
        loss.backward()
        optimizer.step()
        torch.cuda.empty_cache()
        total_loss += loss.item()
    return total_loss / len(labeled_loader)

# ---------- RLE 인코딩 ----------
def mask_to_rle(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(map(str, runs))

# ---------- 메인 ----------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])
    # 데이터로더
    labeled_ds = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/train/train/frames",
        "/kaggle/input/lanesegmentationchallenge/train/train/lane-masks",
        transform
    )
    unlabeled_ds = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/test/test/frames",
        None, transform
    )
    labeled_loader = DataLoader(labeled_ds, batch_size=1, shuffle=True, num_workers=4)
    unlabeled_loader = DataLoader(unlabeled_ds, batch_size=1, shuffle=True, num_workers=4)

    # 모델 및 옵티마이저
    model = SimpleCNNModel().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Semi-supervised 학습
    epochs = 3
    print("▶️ Semi-supervised 학습 시작...")
    for ep in range(epochs):
        loss = train_semi(model, labeled_loader, unlabeled_loader, optimizer, device)
        print(f"Epoch {ep+1}/{epochs} - Loss: {loss:.4f}")
    torch.save(model.state_dict(), "semi_cnn_model.pth")
    print("💾 Semi-supervised 모델 저장 완료")

    # --- 테스트 및 제출 ---
    print("🧪 submission.csv 생성 시작...")
    model.eval()
    results = []
    os.makedirs('pred_masks', exist_ok=True)
    with torch.no_grad():
        for imgs, fnames, orig_size in tqdm(DataLoader(unlabeled_ds, batch_size=1), desc="추론 중"):
            imgs = imgs.to(device)
            output = model(imgs)
            pred_mask = (torch.sigmoid(output).squeeze(0).cpu().numpy() > 0.5).astype(np.uint8)
            W, H = orig_size
            pred_pil = Image.fromarray(pred_mask * 255)
            resized_mask = pred_pil.resize((W, H), resample=Image.NEAREST)
            resized_bin = np.array(resized_mask) > 127
            rle = mask_to_rle(resized_bin.astype(np.uint8))
            # 예측 마스크 저장
            mask_path = os.path.join('pred_masks', fnames[0])
            pred_pil.save(mask_path)
            results.append({'filename': fnames[0], 'rle': rle})
    pd.DataFrame(results).to_csv("submission.csv", index=False)
    print("✅ submission.csv 파일이 저장되었습니다!")

if __name__ == "__main__":
    main()


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 33.12 MiB is free. Process 3330 has 15.85 GiB memory in use. Of the allocated memory 15.57 GiB is allocated by PyTorch, and 1.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [24]:
torch.cuda.empty_cache()

In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# ---------- Dice Loss 함수 정의 ----------
def dice_loss(pred, target, smooth=1e-6):
    prob = torch.sigmoid(pred)
    inter = (prob * target).sum(dim=(1, 2))
    union = prob.sum(dim=(1, 2)) + target.sum(dim=(1, 2))
    dice = (2 * inter + smooth) / (union + smooth)
    return 1 - dice.mean()

# ---------- 데이터셋 클래스 ----------
class LaneDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = sorted(os.listdir(img_dir))
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(os.path.join(self.img_dir, img_name)).convert('RGB')
        img_t = self.transform(img) if self.transform else transforms.ToTensor()(img)
        if self.mask_dir:
            mask = Image.open(os.path.join(self.mask_dir, img_name)).convert('L')
            mask = mask.resize((img_t.shape[2], img_t.shape[1]), Image.NEAREST)
            mask_t = torch.from_numpy((np.array(mask) > 127).astype(np.float32))
            return img_t, mask_t
        else:
            return img_t, img_name, img.size

# ---------- 경량 CNN 모델 정의 ----------
class LightSegNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 인코더
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.enc3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU()
        )
        # 디코더
        self.dec3 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 2, stride=2), nn.ReLU()
        )
        self.dec2 = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 2, stride=2), nn.ReLU()
        )
        self.classifier = nn.Conv2d(16, 1, 1)

    def forward(self, x):
        x1 = self.enc1(x)
        x2 = self.enc2(x1)
        x3 = self.enc3(x2)
        d2 = self.dec3(x3)
        d1 = self.dec2(d2)
        out = self.classifier(d1)
        return out.squeeze(1)

# ---------- Semi-Supervised 학습 함수 ----------
def train_semi(model, labeled_loader, unlabeled_loader, optimizer, device, lambda_u=0.5):
    model.train()
    total_loss = 0.0
    unl_iter = iter(unlabeled_loader)
    bce = nn.BCEWithLogitsLoss()
    for imgs_l, masks in tqdm(labeled_loader, desc='Semi-Train'):
        imgs_l, masks = imgs_l.to(device), masks.to(device)
        out_l = model(imgs_l)
        loss_sup = bce(out_l, masks) + dice_loss(out_l, masks)
        try:
            imgs_u, _, _ = next(unl_iter)
        except StopIteration:
            unl_iter = iter(unlabeled_loader)
            imgs_u, _, _ = next(unl_iter)
        imgs_u = imgs_u.to(device)
        with torch.no_grad():
            pseudo = torch.sigmoid(model(imgs_u))
        mask_p = (pseudo > 0.9).float()
        loss_unsup = bce(model(imgs_u), mask_p)
        loss = loss_sup + lambda_u * loss_unsup
        model.zero_grad()
        loss.backward()
        optimizer.step()
        if device.type == 'cuda':
            torch.cuda.empty_cache()
        total_loss += loss.item()
    return total_loss / len(labeled_loader)

# ---------- RLE 인코딩 ----------
def mask_to_rle(mask):
    pixels = mask.flatten(order='F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(map(str, runs))

# ---------- 메인 ----------
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = transforms.Compose([
        transforms.Resize((256, 512)),
        transforms.ToTensor()
    ])
    # 데이터로더
    labeled_ds = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/train/train/frames",
        "/kaggle/input/lanesegmentationchallenge/train/train/lane-masks",
        transform
    )
    unlabeled_ds = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/test/test/frames",
        None, transform
    )
    labeled_loader = DataLoader(labeled_ds, batch_size=1, shuffle=True, num_workers=2)
    unlabeled_loader = DataLoader(unlabeled_ds, batch_size=1, shuffle=True, num_workers=2)

    # 모델 및 옵티마이저
    model = LightSegNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=5e-4)

    # Semi-supervised 학습
    epochs = 10
    print("▶️ Semi-supervised 학습 시작...")
    for ep in range(1, epochs+1):
        loss = train_semi(model, labeled_loader, unlabeled_loader, optimizer, device)
        print(f"Epoch {ep}/{epochs} - Loss: {loss:.4f}")
    torch.save(model.state_dict(), "semi_lightseg.pth")
    print("💾 Semi-supervised 모델 저장 완료")

    # --- 테스트 및 제출 ---
    print("🧪 submission.csv 생성 시작...")
    model.eval()
    results = []
    os.makedirs('pred_masks', exist_ok=True)
    with torch.no_grad():
        for imgs, fnames, orig_size in tqdm(DataLoader(unlabeled_ds, batch_size=1), desc="추론 중"):
            imgs = imgs.to(device)
            out = model(imgs)
            pred = (torch.sigmoid(out).squeeze(0).cpu().numpy() > 0.5).astype(np.uint8)
            W, H = orig_size
            pil = Image.fromarray(pred * 255)
            pil.save(os.path.join('pred_masks', fnames[0]))
            rle = mask_to_rle(pred)
            results.append({'filename': fnames[0], 'rle': rle})
    pd.DataFrame(results).to_csv("submission.csv", index=False)
    print("✅ submission.csv 파일이 저장되었습니다!")

if __name__ == '__main__':
    main()


▶️ Semi-supervised 학습 시작...


Semi-Train: 100%|██████████| 3626/3626 [01:21<00:00, 44.64it/s]


Epoch 1/10 - Loss: 0.9692


Semi-Train: 100%|██████████| 3626/3626 [01:14<00:00, 48.66it/s]

Epoch 2/10 - Loss: 0.7978


...

Epoch 9/10 - Loss: 0.6082



Semi-Train: 100%|██████████| 3626/3626 [01:15<00:00, 47.98it/s]


Epoch 10/10 - Loss: 0.5978
💾 Semi-supervised 모델 저장 완료
🧪 submission.csv 생성 시작...


추론 중: 100%|██████████| 2782/2782 [00:54<00:00, 50.99it/s]


✅ submission.csv 파일이 저장되었습니다!


In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# ---------- Dice Loss 함수 정의 ----------
def dice_loss(pred, target, smooth=1e-6):
    prob = torch.sigmoid(pred)
    inter = (prob * target).sum(dim=(1, 2))
    union = prob.sum(dim=(1, 2)) + target.sum(dim=(1, 2))
    dice = (2 * inter + smooth) / (union + smooth)
    return 1 - dice.mean()

# ---------- 데이터셋 클래스 ----------
class LaneDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = sorted(os.listdir(img_dir))
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img = Image.open(os.path.join(self.img_dir, img_name)).convert('RGB')
        img_t = self.transform(img) if self.transform else transforms.ToTensor()(img)
        if self.mask_dir:
            mask = Image.open(os.path.join(self.mask_dir, img_name)).convert('L')
            mask = mask.resize((img_t.shape[2], img_t.shape[1]), Image.NEAREST)
            mask_t = torch.from_numpy((np.array(mask) > 127).astype(np.float32))
            return img_t, mask_t
        else:
            return img_t, img_name, img.size

# ---------- 경량 CNN 모델 정의 ----------
class LightSegNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 인코더
        self.enc1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.enc2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.enc3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU()
        )
        # 디코더
        self.dec3 = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 2, stride=2), nn.ReLU()
        )
        self.dec2 = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 2, stride=2), nn.ReLU()
        )
        self.classifier = nn.Conv2d(16, 1, 1)

    def forward(self, x):
        x1 = self.enc1(x)
        x2 = self.enc2(x1)
        x3 = self.enc3(x2)
        d2 = self.dec3(x3)
        d1 = self.dec2(d2)
        out = self.classifier(d1)
        return out.squeeze(1)
def train_supervised(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0.0
    bce = nn.BCEWithLogitsLoss()
    for imgs, masks in tqdm(data_loader, desc='Supervised-Train'):
        imgs, masks = imgs.to(device), masks.to(device)
        out = model(imgs)
        loss = bce(out, masks) + dice_loss(out, masks)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        if device.type == 'cuda':
            torch.cuda.empty_cache()
        total_loss += loss.item()
    return total_loss / len(data_loader)
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = transforms.Compose([
        transforms.Resize((256, 512)),
        transforms.ToTensor()
    ])
    # 데이터로더
    labeled_ds = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/train/train/frames",
        "/kaggle/input/lanesegmentationchallenge/train/train/lane-masks",
        transform
    )
    unlabeled_ds = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/test/test/frames",
        None, transform
    )
    labeled_loader = DataLoader(labeled_ds, batch_size=1, shuffle=True, num_workers=2)
    unlabeled_loader = DataLoader(unlabeled_ds, batch_size=1, shuffle=True, num_workers=2)

    # 모델 및 옵티마이저
    model = LightSegNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=5e-4)

    # Semi-supervised 학습
    epochs = 10
    print("▶️ Semi-supervised 학습 시작...")
    for ep in range(1, epochs+1):
        loss = train_supervised(model, labeled_loader, unlabeled_loader, optimizer, device)
        print(f"Epoch {ep}/{epochs} - Loss: {loss:.4f}")
    torch.save(model.state_dict(), "semi_lightseg.pth")
    print("💾 supervised 모델 저장 완료")

    # --- 테스트 및 제출 ---
    print("🧪 submission.csv 생성 시작...")
    model.eval()
    results = []
    os.makedirs('pred_masks', exist_ok=True)
    with torch.no_grad():
        for imgs, fnames, orig_size in tqdm(DataLoader(unlabeled_ds, batch_size=1), desc="추론 중"):
            imgs = imgs.to(device)
            out = model(imgs)
            pred = (torch.sigmoid(out).squeeze(0).cpu().numpy() > 0.5).astype(np.uint8)
            W, H = orig_size
            pil = Image.fromarray(pred * 255)
            pil.save(os.path.join('pred_masks', fnames[0]))
            rle = mask_to_rle(pred)
            results.append({'filename': fnames[0], 'rle': rle})
    pd.DataFrame(results).to_csv("submission.csv", index=False)
    print("✅ submission.csv 파일이 저장되었습니다!")

if __name__ == '__main__':
    main()